In [1]:
import zipfile

zip_ref = zipfile.ZipFile("all.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [2]:
zip_ref = zipfile.ZipFile("train.zip", 'r')
zip_ref.extractall(".")
zip_ref = zipfile.ZipFile("test.zip", 'r')
zip_ref.extractall(".")
zip_ref.close()

In [3]:
import os
import shutil

train_filenames = os.listdir('train')
train_cat = filter(lambda x:x[:3] == 'cat', train_filenames)
train_dog = filter(lambda x:x[:3] == 'dog', train_filenames)

In [4]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

In [5]:
rmrf_mkdir('trainRun')
os.mkdir('trainRun/cat')
os.mkdir('trainRun/dog')

rmrf_mkdir('testRun')
os.symlink('../test/', 'testRun/test')

In [6]:
for filename in train_cat:
    os.symlink('../../train/'+filename, 'trainRun/cat/'+filename)

for filename in train_dog:
    os.symlink('../../train/'+filename, 'trainRun/dog/'+filename)
    

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

import h5py

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_model = InceptionV3(weights='imagenet', include_top=False)

In [3]:
# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [4]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [5]:
train_data_dir = 'trainRun'
train_datagen = image.ImageDataGenerator(rescale=1./255,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True)

In [6]:
img_width, img_height = 299, 299
nm_batch_size = 16
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    shuffle=False,
    batch_size=nm_batch_size,
    class_mode='binary')

Found 25000 images belonging to 2 classes.


In [7]:
train_h5py = model.predict_generator(train_generator)

In [8]:
test_data_dir = 'testRun'
test_datagen = image.ImageDataGenerator(rescale=1./255)

In [9]:
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(img_height, img_width),
                                                 shuffle=False,
                                                 batch_size=nm_batch_size,
                                                 class_mode=None)

Found 12500 images belonging to 1 classes.


In [11]:
test_h5py = model.predict_generator(test_generator)

In [13]:
with h5py.File("init_weights_InceptionV3.h5") as h:
        h.create_dataset("train", data=train_h5py)
        h.create_dataset("test", data=test_h5py)
        h.create_dataset("label", data=train_generator.classes)

In [15]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2017)

In [16]:
X_train = []
X_test = []

In [17]:
with h5py.File("init_weights_InceptionV3.h5", 'r') as h:
    X_train.append(np.array(h['train']))
    X_test.append(np.array(h['test']))
    y_train = np.array(h['label'])

In [18]:
X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

In [19]:
X_train, y_train = shuffle(X_train, y_train)

In [20]:
from keras.layers import Input, Dropout

In [21]:
np.random.seed(2017)

In [22]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model_run = Model(input_tensor, x)

model_run.compile(optimizer='adadelta',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])

In [23]:
model_run.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Train on 20000 samples, validate on 5000 samples
Epoch 1/8
20000/20000 [==============================] - 1s 71us/step - loss: 0.6924 - acc: 0.5657 - val_loss: 0.6918 - val_acc: 0.8520
Epoch 2/8
20000/20000 [==============================] - 1s 27us/step - loss: 0.6912 - acc: 0.6383 - val_loss: 0.6905 - val_acc: 0.8116
Epoch 3/8
20000/20000 [==============================] - 1s 26us/step - loss: 0.6899 - acc: 0.7096 - val_loss: 0.6892 - val_acc: 0.7804
Epoch 4/8
20000/20000 [==============================] - 1s 27us/step - loss: 0.6887 - acc: 0.7436 - val_loss: 0.6880 - val_acc: 0.9014
Epoch 5/8
20000/20000 [==============================] - 1s 27us/step - loss: 0.6876 - acc: 0.7577 - val_loss: 0.6869 - val_acc: 0.9474
Epoch 6/8
20000/20000 [==============================] - 1s 26us/step - loss: 0.6863 - acc: 0.7775 - val_loss: 0.6856 - val_acc: 0.9528
Epoch 7/8
20000/20000 [==============================] - 1s 27us/step - loss: 0.6851 - acc: 0.7760 - val_loss: 0.6844 - val_acc: 0.9658

In [24]:
y_pred = model_run.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.003, max=0.997)

12500/12500 [==============================] - 1s 51us/step


In [25]:
import pandas as pd

df = pd.read_csv("sample_submission.csv")

test_run_datagen = image.ImageDataGenerator(rescale=1./255)
test_run_generator = test_run_datagen.flow_from_directory("testRun", 
                                                          (img_height, img_width), 
                                                         shuffle=False, 
                                                         batch_size=nm_batch_size, 
                                                         class_mode=None)


Found 12500 images belonging to 1 classes.


In [26]:
for i, fname in enumerate(test_run_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [28]:
df.to_csv('pred.csv', index=None)
df.head(10)

id     label
0   1  0.501860
1   2  0.506907
2   3  0.505850
3   4  0.505536
4   5  0.494151
5   6  0.493319
6   7  0.494156
7   8  0.495172
8   9  0.491440
9  10  0.493716